In [1]:
from pyspark.sql import Row
from pyspark.sql import functions as F

In [2]:

Employee = Row('employee1', 'employee2', 'employee3', 'employee4', 'employee5')
Salaries = Row('100000', '120000', '140000', '160000', '160000')

departmentWithEmployees1 = Row(employees=[Employee, Salaries])

departmentsWithEmployees_Seq = [departmentWithEmployees1]
dframe = spark.createDataFrame(departmentsWithEmployees_Seq)
dframe.show(truncate=False)
dframe.printSchema()

+---------------------------------------------------------------------------------------------------+
employees |
+---------------------------------------------------------------------------------------------------+
[[employee1, employee2, employee3, employee4, employee5], [100000, 120000, 140000, 160000, 160000]]|
+---------------------------------------------------------------------------------------------------+

root
-- employees: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- _1: string (nullable = true)
 | |-- _2: string (nullable = true)
 | |-- _3: string (nullable = true)
 | |-- _4: string (nullable = true)
 | |-- _5: string (nullable = true)

In [3]:
dframe.withColumn("firstname", F.element_at("employees", 1))\
.withColumn("salary",F.element_at("employees",2))\
.drop("employees")\
.withColumn("firstname",F.array("firstname.*"))\
.withColumn("salary", F.array("salary.*"))\
.withColumn("zip",F.explode(F.arrays_zip("firstname","salary")))\
.select("zip.*").show(truncate=False)

+---------+------+
firstname|salary|
+---------+------+
employee1|100000|
employee2|120000|
employee3|140000|
employee4|160000|
employee5|160000|
+---------+------+

In [4]:
dframe.withColumn("firstname", F.element_at("employees", 1))\
.withColumn("salary",F.element_at("employees",2)).drop("employees")\
.withColumn("zip",F.explode(F.arrays_zip((F.array("firstname.*").alias("firstname")),(F.array("salary.*").alias("salary")))))\
.select("zip.*").show(truncate=False)

+---------+------+
0 |1 |
+---------+------+
employee1|100000|
employee2|120000|
employee3|140000|
employee4|160000|
employee5|160000|
+---------+------+

In [5]:
.withColumn("firstname",F.array("firstname.*").printSchema()

In [6]:
dframe.withColumn("employees",F.explode("employees")).show(truncate=False)

+-------------------------------------------------------+
employees |
+-------------------------------------------------------+
[employee1, employee2, employee3, employee4, employee5]|
[100000, 120000, 140000, 160000, 160000] |
+-------------------------------------------------------+

In [7]:

list=[[1,10,50,12],
      [3,40,50,12],
      [4,20,60,15],
      [6,80,80,18]]


df=spark.createDataFrame(list,['No','A','B','C'])
df.show()

+---+---+---+---+
 No| A| B| C|
+---+---+---+---+
 1| 10| 50| 12|
 3| 40| 50| 12|
 4| 20| 60| 15|
 6| 80| 80| 18|
+---+---+---+---+

In [8]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import when
w=Window().partitionBy().orderBy("No")
w2=Window().partitionBy().orderBy("No").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("number2", F.lag("No").over(w)).withColumn("diff", F.when((F.col("number2").isNotNull()) & ((F.col("No")-F.col("number2")) > 1), (F.col("No")-F.col("number2"))).otherwise(F.lit(0)))\
.withColumn("diff2", F.lead("diff").over(w)).withColumn("diff2", F.when(F.col("diff2").isNull(), F.lit(0)).otherwise(F.col("diff2"))).withColumn("diff2", F.when(F.col("diff2")!=0, F.col("diff2")-1).otherwise(F.col("diff2"))).withColumn("max", F.max("No").over(w2))\
.withColumn("diff2", F.when((F.col("No")==F.col("max")) & (F.col("No")<F.col("max"), F.col("max")-F.col("No"))).otherwise(F.col("diff2")))\
.withColumn("number2", F.when(F.col("diff2")!=0,F.expr("""sequence(No,No+diff2,1)""")).otherwise(F.expr("""sequence(No,No+diff2,0)""")))\
.drop("diff","diff2","max")\
.withColumn("number2", F.explode("number2")).drop("No")\
.select(F.col("number2").alias("No"), "A","B","C")\
.show()

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-729350543556944> in <module> 
 6 df . withColumn ( "number2" , F . lag ( "No" ) . over ( w ) ) . withColumn ( "diff" , F . when ( ( F . col ( "number2" ) . isNotNull ( ) ) & ( ( F . col ( "No" ) - F . col ( "number2" ) ) > 1 ) , ( F . col ( "No" ) - F . col ( "number2" ) ) ) . otherwise ( F . lit ( 0 ) ) ) \ 
 7 . withColumn ( "diff2" , F . lead ( "diff" ) . over ( w ) ) . withColumn ( "diff2" , F . when ( F . col ( "diff2" ) . isNull ( ) , F . lit ( 0 ) ) . otherwise ( F . col ( "diff2" ) ) ) . withColumn ( "diff2" , F . when ( F . col ( "diff2" ) != 0 , F . col ( "diff2" ) - 1 ) . otherwise ( F . col ( "diff2" ) ) ) . withColumn ( "max" , F . max ( "No" ) . over ( w2 ) ) \ 
 ----> 8 . withColumn ( "diff2" , F . when ( ( F . col ( "No" ) == F . col ( "max" ) ) & ( F . col ( "No" ) < F . col ( "max" ) , F . col ( "max" ) - F . col ( "No" ) ) ) . otherwise ( F . col ( "diff2" ) ) ) \ 
 9 . withColumn ( "number2" , F . when ( F . col ( "diff2" ) != 0 , F . expr ( """sequence(No,No+diff2,1)""" ) ) . otherwise ( F . expr ( """sequence(No,No+diff2,0)""" ) ) ) \ 
 10 . drop ( "diff" , "diff2" , "max" ) \ 

 /databricks/spark/python/pyspark/sql/column.py in _ (self, other) 
 113 def _ ( self , other ) : 
 114 jc = other . _jc if isinstance ( other , Column ) else other
 --> 115 njc = getattr ( self . _jc , name ) ( jc ) 
 116 return Column ( njc ) 
 117 _ . __doc__ = doc

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 499 java_list = ArrayList ( ) 
 500 for element in object : 
 --> 501 java_list . add ( element ) 
 502 return java_list
 503 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1246 
 1247 def __call__ ( self , * args ) : 
 -> 1248 args_command , temp_args = self . _build_args ( * args ) 
 1249 
 1250 command = proto . CALL_COMMAND_NAME + \ 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _build_args (self, *args) 
 1210 def _build_args ( self , * args ) : 
 1211 if self . converters is not None and len ( self . converters ) > 0 : 
 -> 1212 ( new_args , temp_args ) = self . _get_args ( args ) 
 1213 else : 
 1214 new_args = args

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in _get_args (self, args) 
 1197 for converter in self . gateway_client . converters : 
 1198 if converter . can_convert ( arg ) : 
 -> 1199 temp_arg = converter . convert ( arg , self . gateway_client ) 
 1200 temp_args . append ( temp_arg ) 
 1201 new_args . append ( temp_arg ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_collections.py in convert (self, object, gateway_client) 
 498 ArrayList = JavaClass ( "java.util.ArrayList" , gateway_client ) 
 499 java_list = ArrayList ( ) 
 --> 500 for element in object : 
 501 java_list . add ( element ) 
 502 return java_list

 /databricks/spark/python/pyspark/sql/c

In [9]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import when
w=Window().partitionBy().orderBy("No")
w2=Window().partitionBy().orderBy("No").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("number2", F.lag("No").over(w)).withColumn("diff", F.when((F.col("number2").isNotNull()) & ((F.col("No")-F.col("number2")) > 1), (F.col("No")-F.col("number2"))).otherwise(F.lit(0)))\
.withColumn("diff2", F.lead("diff").over(w)).withColumn("diff2", F.when(F.col("diff2").isNull(), F.lit(0)).otherwise(F.col("diff2"))).withColumn("diff2", F.when(F.col("diff2")!=0, F.col("diff2")-1).otherwise(F.col("diff2"))).withColumn("max", F.max("No").over(w2))\
.withColumn("diff2", F.when((F.col("No")==F.col("max")) & (F.col("No")<F.col("max")), F.col("max")-F.col("No")).otherwise(F.col("diff2")))\
.withColumn("number2", F.when(F.col("diff2")!=0,F.expr("""sequence(No,No+diff2,1)""")).otherwise(F.expr("""sequence(No,No+diff2,0)""")))\
.drop("diff","diff2")\
.withColumn("number2", F.explode("number2")).drop("No")\
.select(F.col("number2").alias("No"), "A","B","C")\
.show()

+---+---+---+---+
 No| A| B| C|
+---+---+---+---+
 1| 10| 50| 12|
 2| 10| 50| 12|
 3| 40| 50| 12|
 4| 20| 60| 15|
 5| 20| 60| 15|
 6| 80| 80| 18|
+---+---+---+---+

In [10]:
list = [['A','B','C','col2,col3','X,Y'],
       ['P','Q','R','col1','Z'],
       ['I','J','K','col1,col3','S,T']]
df=spark.createDataFrame(list,['col1','col2','col3','col4','col5'])
df.show()

+----+----+----+---------+----+
col1|col2|col3| col4|col5|
+----+----+----+---------+----+
 A| B| C|col2,col3| X,Y|
 P| Q| R| col1| Z|
 I| J| K|col1,col3| S,T|
+----+----+----+---------+----+

In [11]:
list=  [[673895,35578,20180101,1,None], 
        [673895,35578,20180102,0,110],  
        [673895,35578,20180103,1,None],     
        [673895,35578,20180104,0,None],    
        [673895,35578,20180105,0,  109], 
        [673895,35578,20180106,1,None], 
        [673895,35578,20180107,0,  108],    
        [673895,35578,20180108,0,None],   
        [673895,35578,20180109,0,None],    
        [673895,35578,20180110,1,None]] 

df=spark.createDataFrame(list,['item','store','timestamp','sales_qty','stock'])
df.show()

+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| null|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| null|
673895|35578| 20180104| 0| null|
673895|35578| 20180105| 0| 109|
673895|35578| 20180106| 1| null|
673895|35578| 20180107| 0| 108|
673895|35578| 20180108| 0| null|
673895|35578| 20180109| 0| null|
673895|35578| 20180110| 1| null|
+------+-----+---------+---------+-----+

In [12]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
w=Window().partitionBy("item","store").orderBy("timestamp")
w2=Window().partitionBy("item","store").orderBy("timestamp").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("stock1", F.when(F.col("stock").isNull(), F.lit(0)).otherwise(F.col("stock")))\
.withColumn("stock2", F.sum("sales_qty").over(w)- F.lit(1))\
.withColumn("first", F.first("stock", True).over(w2))\
.withColumn("stock", F.col("first")-F.col("stock2"))\
.drop("stock1","stock2","first")\
.show()

+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| 110|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| 109|
673895|35578| 20180104| 0| 109|
673895|35578| 20180105| 0| 109|
673895|35578| 20180106| 1| 108|
673895|35578| 20180107| 0| 108|
673895|35578| 20180108| -4| 112|
673895|35578| 20180109| 0| 112|
673895|35578| 20180110| 1| 111|
+------+-----+---------+---------+-----+

In [13]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
w=Window().partitionBy("item","store").orderBy("timestamp")
w2=Window().partitionBy("item","store").orderBy("timestamp").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("stock1", F.when(F.col("stock").isNull(), F.lit(0)).otherwise(F.col("stock")))\
.withColumn("stock2", F.sum("sales_qty").over(w)- F.lit(1))\
.withColumn("first", F.first("stock", True).over(w2))\
.withColumn("stock", F.when(F.col("sales_qty")<0, (F.col("stock2")-F.col("first"))*-1).otherwise(F.col("first")-F.col("stock2")))\
.withColumn("num", F.row_number().over(w))\
.withColumn("stock", F.when(F.col("num")==1, F.lit(None)).otherwise(F.col("stock")))\
.drop("stock1","stock2","first","num")\
.show()

+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| null|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| 109|
673895|35578| 20180104| 0| 109|
673895|35578| 20180105| 0| 109|
673895|35578| 20180106| 1| 108|
673895|35578| 20180107| 0| 108|
673895|35578| 20180108| -4| 112|
673895|35578| 20180109| 0| 112|
673895|35578| 20180110| 1| 111|
+------+-----+---------+---------+-----+

In [14]:
list=  [[673895,35578,20180101,1,None], 
        [673895,35578,20180102,0,110],  
        [673895,35578,20180103,1,None],     
        [673895,35578,20180104,7,None],    
        [673895,35578,20180105,0,  112], 
        [673895,35578,20180106,2,None], 
        [673895,35578,20180107,0,  107],    
        [673895,35578,20180108,0,None],   
        [673895,35578,20180109,0,None],    
        [673895,35578,20180110,1,None]] 

from pyspark.sql.window import Window
from pyspark.sql import functions as F
df=spark.createDataFrame(list,['item','store','timestamp','sales_qty','stock'])
df.show()

+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| null|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| null|
673895|35578| 20180104| 7| null|
673895|35578| 20180105| 0| 112|
673895|35578| 20180106| 2| null|
673895|35578| 20180107| 0| 107|
673895|35578| 20180108| 0| null|
673895|35578| 20180109| 0| null|
673895|35578| 20180110| 1| null|
+------+-----+---------+---------+-----+

In [15]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
w=Window().partitionBy("item","store").orderBy("timestamp")
w2=Window().partitionBy("item","store").orderBy("timestamp").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("stock1", F.when(F.col("stock").isNull(), F.lit(0)).otherwise(F.col("stock")))\
.withColumn("stock2", F.sum("sales_qty").over(w)- F.lit(1))\
.withColumn("first", F.first("stock", True).over(w2))\
.withColumn("stock", F.col("first")-F.col("stock2"))\
.withColumn("num", F.row_number().over(w))\
.withColumn("stock", F.when(F.col("num")==1, F.lit(None)).otherwise(F.col("stock")))\
.drop("stock1","stock2","first","num")\
.show()

+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| null|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| 109|
673895|35578| 20180104| 7| 102|
673895|35578| 20180105| 0| 102|
673895|35578| 20180106| 0| 102|
673895|35578| 20180107| 4| 98|
673895|35578| 20180108| 0| 98|
673895|35578| 20180109| 0| 98|
673895|35578| 20180110| 1| 97|
+------+-----+---------+---------+-----+

In [16]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
w=Window().partitionBy("item","store").orderBy("timestamp")
w2=Window().partitionBy("item","store").orderBy("timestamp").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
df.withColumn("stock1", F.when(F.col("stock").isNull(), F.lit(0)).otherwise(F.col("stock")))\
.withColumn("stock2", F.sum("sales_qty").over(w)- F.lit(1))\
.withColumn("first", F.first("stock", True).over(w2))\
.withColumn("stock", F.col("first")-F.col("stock2"))\
.withColumn("num", F.row_number().over(w))\
.withColumn("stock", F.when(F.col("num")==1, F.lit(None)).otherwise(F.col("stock")))\
.drop("stock1","stock2","first","num")\
.show()

+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| null|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| 109|
673895|35578| 20180104| 0| 109|
673895|35578| 20180105| 0| 109|
673895|35578| 20180106| 4| 105|
673895|35578| 20180107| 0| 105|
673895|35578| 20180108| 4| 101|
673895|35578| 20180109| 0| 101|
673895|35578| 20180110| 1| 100|
+------+-----+---------+---------+-----+

In [17]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

df.show()
w2=Window().partitionBy("item","store")
w=Window().partitionBy("item","store").orderBy("timestamp")
w3=Window().partitionBy("item","store","stock5").orderBy("timestamp")
df.withColumn("stock1", F.when(F.col("stock").isNull(), F.lit(0)).otherwise(F.col("stock")))\
.withColumn("stock9", F.sum("stock").over(w2))\
.withColumn("stock4", F.when(F.col("stock1")!=0, F.dense_rank().over(w)).otherwise(F.col("stock1")))\
.withColumn("stock5", F.sum("stock4").over(w))\
.withColumn("stock6", F.sum("stock1").over(w3))\
.withColumn("sum", F.sum(F.when(F.col("stock1")!=F.col("stock6"),F.col("sales_qty")).otherwise(F.lit(0))).over(w3))\
.withColumn("stock2", F.when(F.col("sales_qty")!=0, F.col("stock6")-F.col("sum")).otherwise(F.col("stock")))\
.withColumn("stock", F.when((F.col("stock2").isNull())&(F.col("sales_qty")==0),F.col("stock6")-F.col("sum")).otherwise(F.col("stock2")))\
.show()


+------+-----+---------+---------+-----+
 item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
673895|35578| 20180101| 1| null|
673895|35578| 20180102| 0| 110|
673895|35578| 20180103| 1| null|
673895|35578| 20180104| 7| null|
673895|35578| 20180105| 0| 112|
673895|35578| 20180106| 2| null|
673895|35578| 20180107| 0| 107|
673895|35578| 20180108| 0| null|
673895|35578| 20180109| 0| null|
673895|35578| 20180110| 1| null|
+------+-----+---------+---------+-----+

+------+-----+---------+---------+-----+------+------+------+------+------+---+------+
 item|store|timestamp|sales_qty|stock|stock1|stock9|stock4|stock5|stock6|sum|stock2|
+------+-----+---------+---------+-----+------+------+------+------+------+---+------+
673895|35578| 20180101| 1| 0| 0| 329| 0| 0| 0| 0| 0|
673895|35578| 20180102| 0| 110| 110| 329| 2| 2| 110| 0| 110|
673895|35578| 20180103| 1| 109| 0| 329| 0| 2| 110| 1| 109|
673895|35578| 20180104| 7| 102| 0| 329| 0| 2| 110| 8| 102|
673895|35578| 20180105| 0| 112| 112| 329| 5| 7| 112| 0| 112|
673895|35578| 20180106| 2| 110| 0| 329| 0| 7| 112| 2| 110|
673895|35578| 20180107| 0| 107| 107| 329| 7| 14| 107| 0| 107|
673895|35578| 20180108| 0| 107| 0| 329| 0| 14| 107| 0| null|
673895|35578| 20180109| 0| 107| 0| 329| 0| 14| 107| 0| null|
673895|35578| 20180110| 1| 106| 0| 329| 0| 14| 107| 1| 106|
+------+-----+---------+---------+-----+------+------+------+------+------+---+------+

In [18]:
.when((F.col("stock6")!=0)&(F.col("stock6")!=F.col("stock1")),F.col("stock6")-F.col("sales_qty"))

File "<command-1887727080535783>" , line 1 
 .when((F.col("stock6")!=0)&(F.col("stock6")!=F.col("stock1")),F.col("stock6")-F.col("sales_qty")) 
 ^
 SyntaxError : invalid syntax

In [19]:
   data:                                        expected output:       
   +------+-----+---------+---------+-----+     +------+-----+---------+---------+-----+
   |  item|store|timestamp|sales_qty|stock|     |  item|store|timestamp|sales_qty|stock|
   +------+-----+---------+---------+-----+     +------+-----+---------+---------+-----+
   |673895|35578| 20180101|        1| null|     |673895|35578| 20180101|        1| null|
   |673895|35578| 20180102|        0|  110|     |673895|35578| 20180102|        0|  110|
   |673895|35578| 20180103|        1| null|     |673895|35578| 20180103|        1|  109|
   |673895|35578| 20180104|        0| null|     |673895|35578| 20180104|        0|  109|
   |673895|35578| 20180105|        0|  109|  => |673895|35578| 20180105|        0|  109|
   |673895|35578| 20180106|        1| null|     |673895|35578| 20180106|        1|  108|
   |673895|35578| 20180107|        0|  108|     |673895|35578| 20180107|        0|  108|
   |673895|35578| 20180108|        0| null|     |673895|35578| 20180108|        0|  108|
   |673895|35578| 20180109|        0| null|     |673895|35578| 20180109|        0|  108|
   |673895|35578| 20180110|        1| null|     |673895|35578| 20180110|        1|  107|
   +------+-----+---------+---------+-----+     +------+-----+---------+---------+-----+

In [20]:
+------+-----+---------+---------+-----+
|  item|store|timestamp|sales_qty|stock|
+------+-----+---------+---------+-----+
|673895|35578| 20180101|        1| null|
|673895|35578| 20180102|        0|  110|
|673895|35578| 20180103|        1| null|
|673895|35578| 20180104|        7| null|
|673895|35578| 20180105|        0|  112|
|673895|35578| 20180106|        0| null|
|673895|35578| 20180107|        4|   98|
|673895|35578| 20180108|        0| null|
|673895|35578| 20180109|        0| null|
|673895|35578| 20180110|        1| null|
+------+-----+---------+---------+-----+

